Task 3

In [1]:
import pandas as pd
import plotly.express as px
from plotly.subplots import make_subplots
import plotly.graph_objects as go


In [2]:
# Load dataset
df = pd.read_csv('/content/drive/MyDrive/Grepsr/Dataset - Junior Data QA Analyst.csv', encoding='utf-8')
df['age'] = 2025 - df['dob']  # Calculate age
print(df.shape)

(4335311, 17)


In [3]:
# Summary Statistics
total_rows = len(df)
duplicates = len(df) - len(df.drop_duplicates())
duplicates_key = len(df) - len(df.drop_duplicates(['post_code', 'dob', 'name']))

In [4]:
# Field Summary (Available/Missing Counts)
missing_summary = df.isnull().sum().reset_index()
missing_summary.columns = ['Field', 'Missing Count']
missing_summary['Available Count'] = total_rows - missing_summary['Missing Count']
missing_summary['Available %'] = (missing_summary['Available Count'] / total_rows * 100).round(2)
missing_summary['Missing %'] = (missing_summary['Missing Count'] / total_rows * 100).round(2)
missing_summary['Class'] = missing_summary['Missing %'].apply(lambda x: 'missing-high' if x > 50 else 'complete')

In [5]:
# Provider and Postcode Counts
provider_counts = df['name'].value_counts().reset_index()
provider_counts.columns = ['Provider', 'Count']
postcode_counts = df['post_code'].value_counts().reset_index()
postcode_counts.columns = ['Post Code', 'Count']

In [6]:
# Convert to HTML with commas
missing_summary_html = missing_summary.to_html(index=False, classes='table', escape=False, formatters={
    'Available Count': '{:,.0f}'.format,
    'Missing Count': '{:,.0f}'.format,
    'Available %': lambda x: f'<span class="{missing_summary.loc[missing_summary["Available %"] == x, "Class"].iloc[0]}">{x}%</span>',
    'Missing %': lambda x: f'<span class="{missing_summary.loc[missing_summary["Missing %"] == x, "Class"].iloc[0]}">{x}%</span>'
})
provider_counts_html = provider_counts.to_html(index=False, classes='table', formatters={
    'Count': '{:,.0f}'.format
})
postcode_counts_html = postcode_counts.to_html(index=False, classes='table', formatters={
    'Count': '{:,.0f}'.format
})

In [7]:
# Charts
fig1 = px.histogram(df, x='age', title='Age Distribution', nbins=50, color_discrete_sequence=['#1a73e8'])
fig2 = px.bar(postcode_counts.head(20), x='Post Code', y='Count', title='Top 20 Postcodes by Record Count', color='Count', color_continuous_scale='Viridis')
fig2.update_coloraxes(colorbar_title='Record Count')
fig3 = px.bar(missing_summary, x='Field', y='Missing Count', title='Missing Values per Field', color='Missing %', color_continuous_scale='Reds')
fig3.update_coloraxes(colorbar_title='Missing Percentage (%)')

In [8]:
# Combine Charts
fig = make_subplots(rows=3, cols=1, subplot_titles=['Age Distribution', 'Top 20 Postcodes', 'Missing Values'])
fig.add_trace(fig1.data[0], row=1, col=1)
fig.add_trace(fig2.data[0], row=2, col=1)
fig.add_trace(fig3.data[0], row=3, col=1)
fig.update_layout(height=1200, showlegend=False)

Output hidden; open in https://colab.research.google.com to view.

In [9]:
# Embed Plotly HTML
plotly_html = fig.to_html(full_html=False, include_plotlyjs='cdn')

In [10]:
# Generate HTML Report
html_content = f"""
<html>
<head>
    <title>Data QA Visualization</title>
    <script src="https://cdn.plot.ly/plotly-latest.min.js"></script>
    <style>
        body {{
            font-family: Arial, sans-serif;
            margin: 0;
            background-color: #f5f5f5;
        }}
        .navbar {{
            background-color: #1a73e8;
            overflow: hidden;
            position: sticky;
            top: 0;
            width: 100%;
            z-index: 1000;
        }}
        .navbar a {{
            float: left;
            display: block;
            color: white;
            text-align: center;
            padding: 14px 20px;
            text-decoration: none;
        }}
        .navbar a:hover {{
            background-color: #0f4f9f;
        }}
        h1, h2 {{
            color: #1a73e8;
            margin-top: 20px;
            padding-left: 20px;
        }}
        table {{
            border-collapse: collapse;
            width: calc(100% - 40px); /* Adjust for padding */
            margin: 0 20px 20px 20px; /* Adjust for padding */
            background-color: #fff;
            box-shadow: 0 2px 4px rgba(0,0,0,0.1);
        }}
        th {{
            background-color: #1a73e8;
            color: white;
            padding: 10px;
            text-align: left;
        }}
        td {{
            padding: 8px;
            border: 1px solid #ddd;
        }}
        tr:nth-child(even) {{
            background-color: #f9f9f9;
        }}
        .missing-high {{
            color: #d32f2f;
            font-weight: bold;
        }}
        .complete {{
            color: #388e3c;
        }}
        .description, .explanation {{
            background-color: #e8f0fe;
            padding: 15px;
            border-radius: 5px;
            margin: 20px;
        }}
        .content-section {{
            padding-top: 60px; /* Add padding to account for sticky navbar */
            margin-top: -60px; /* Negative margin to bring content up */
        }}
    </style>
</head>
<body>
    <div class="navbar">
        <a href="#overview">Overview</a>
        <a href="#summary">Summary</a>
        <a href="#provider-counts">Provider Counts</a>
        <a href="#postcode-counts">Postcode Counts</a>
        <a href="#field-summary">Field Summary</a>
        <a href="#visualizations">Visualizations</a>
    </div>

    <div id="overview" class="content-section description">
        <h2>Overview</h2>
        <p>This report presents my analysis of a dataset containing {total_rows:,} records. I evaluated key fields such as age, postcodes, gender, and other attributes to identify anomalies, missing values, and distribution patterns. The report shows invalid ages, uneven postcode distributions, and incomplete fields through styled tables and interactive charts with color bars that indicate value ranges.</p>
    </div>

    <div id="summary" class="content-section">
        <h2>Summary</h2>
        <table>
            <tr><th>Metric</th><th>Value</th></tr>
            <tr><td>Total Row Count</td><td>{total_rows:,}</td></tr>
            <tr><td>Line-wise Duplicates</td><td>{duplicates:,}</td></tr>
            <tr><td>Duplicates (post_code, dob, name)</td><td>{duplicates_key:,}</td></tr>
        </table>
    </div>

    <div id="provider-counts" class="content-section">
        <h2>Provider Counts</h2>
        <table>
            {provider_counts_html}
        </table>
    </div>

    <div id="postcode-counts" class="content-section">
        <h2>Postcode Counts</h2>
        <table>
            {postcode_counts_html}
        </table>
    </div>

    <div id="field-summary" class="content-section">
        <h2>Field Summary</h2>
        <table>
            {missing_summary_html}
        </table>
    </div>

    <div id="visualizations" class="content-section">
        <h2>Visualizations</h2>
        {plotly_html}
    </div>

</body>
</html>
"""

In [11]:
# Save HTML
with open('/content/drive/MyDrive/Grepsr/Task 3/visualization.html', 'w', encoding='utf-8') as f:
    f.write(html_content)